# B1. SharePoint Indexer - セットアップと初回実行

## 概要

このノートブックでは、Azure AI Search の SharePoint インデクサを使った自動取り込みを設定します:

1. **インデックス作成**: 検索用インデックスの定義
2. **データソース作成**: SharePoint Online への接続設定
3. **スキルセット作成**: テキスト抽出・OCR・ベクター化（任意）
4. **インデクサ作成**: 自動取り込みパイプラインの構成
5. **初回実行**: インデクサの起動と確認

## 前提条件

- `00_prereqs_and_env.ipynb` が完了していること
- Azure AI Search サービスが Basic 以上のプランであること
- SharePoint Online サイトとドキュメントライブラリが準備されていること
- SharePoint データソース用の認証情報（アプリ ID とシークレット）

## SharePoint インデクサのメリット

- ✅ **自動化**: コードなしで設定だけで取り込み可能
- ✅ **スケジュール実行**: 定期的な自動同期
- ✅ **組み込みスキル**: OCR、テキスト抽出が標準装備
- ✅ **保守性**: 運用が容易

## 1. 環境の初期化

In [ ]:
import os
import json
from pathlib import Path
from dotenv import load_dotenv
from typing import Dict, List, Optional
import time

from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient, SearchIndexerClient
from azure.search.documents.indexes.models import (
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    AzureOpenAIVectorizer,
    AzureOpenAIVectorizerParameters,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField,
    SemanticSearch,
    SearchIndexerDataSourceConnection,
    SearchIndexerDataContainer,
    SearchIndexer,
    IndexingSchedule,
    SearchIndexerSkillset,
    InputFieldMappingEntry,
    OutputFieldMappingEntry,
    SplitSkill,
    OcrSkill,
    MergeSkill,
    AzureOpenAIEmbeddingSkill,
    FieldMapping
)

# 環境変数の読み込み
env_path = Path("../.env")
if env_path.exists():
    load_dotenv(env_path)
    print("✅ .env ファイルを読み込みました")
else:
    print("⚠️ .env ファイルが見つかりません")

# 必須環境変数の確認
required_vars = ["AZURE_SEARCH_ENDPOINT", "AZURE_SEARCH_API_KEY"]
missing_vars = [var for var in required_vars if not os.getenv(var)]
if missing_vars:
    raise ValueError(f"環境変数が設定されていません: {', '.join(missing_vars)}")

print("\n✅ 環境変数の確認完了")

## 2. 設定パラメータ

In [ ]:
# Azure AI Search 設定
search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
search_key = os.getenv("AZURE_SEARCH_API_KEY")
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME", "sp-docs-indexer")

# SharePoint 設定
sp_hostname = os.getenv("SP_HOSTNAME", "yourtenant.sharepoint.com")
sp_site_path = os.getenv("SP_SITE_PATHS", "/sites/ai-search-handson")
sp_library = os.getenv("SP_LIBRARY_NAMES", "Documents")

# SharePoint データソース認証
# アプリ登録の情報が必要
sp_tenant_id = os.getenv("GRAPH_TENANT_ID", "")
sp_client_id = os.getenv("GRAPH_CLIENT_ID", "")
sp_client_secret = os.getenv("GRAPH_CLIENT_SECRET", "")

# Azure OpenAI 設定（ベクター化用、オプション）
openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT", "")
openai_key = os.getenv("AZURE_OPENAI_API_KEY", "")
embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-3-large")

# インデクサ設定
datasource_name = f"{index_name}-datasource"
skillset_name = f"{index_name}-skillset"
indexer_name = f"{index_name}-indexer"

# ベクター検索を使用するか
use_vector_search = bool(openai_endpoint and openai_key)

print(f"✅ 設定パラメータ:")
print(f"   検索エンドポイント: {search_endpoint}")
print(f"   インデックス名: {index_name}")
print(f"   SharePoint サイト: https://{sp_hostname}{sp_site_path}")
print(f"   ライブラリ: {sp_library}")
print(f"   ベクター検索: {'有効' if use_vector_search else '無効'}")

if not sp_client_id or not sp_client_secret:
    print("\n⚠️ SharePoint データソース用の認証情報が設定されていません")
    print("   .env に GRAPH_CLIENT_ID と GRAPH_CLIENT_SECRET を設定してください")

## 3. クライアントの初期化

In [ ]:
# 認証情報
credential = AzureKeyCredential(search_key)

# インデックス管理クライアント
index_client = SearchIndexClient(
    endpoint=search_endpoint,
    credential=credential
)

# インデクサ管理クライアント
indexer_client = SearchIndexerClient(
    endpoint=search_endpoint,
    credential=credential
)

print("✅ クライアント作成完了")

## 4. インデックスの作成

SharePoint ドキュメント用のインデックススキーマを定義します。

In [ ]:
# インデックスフィールドの定義
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, filterable=True),
    SearchableField(name="title", type=SearchFieldDataType.String, sortable=True),
    SearchableField(name="content", type=SearchFieldDataType.String),
    SimpleField(name="url", type=SearchFieldDataType.String, filterable=True),
    SimpleField(name="path", type=SearchFieldDataType.String, filterable=True),
    SimpleField(name="contentType", type=SearchFieldDataType.String, filterable=True),
    SimpleField(name="lastModified", type=SearchFieldDataType.DateTimeOffset, sortable=True, filterable=True),
    SimpleField(name="author", type=SearchFieldDataType.String, filterable=True),
    SimpleField(name="size", type=SearchFieldDataType.Int64, sortable=True),
    SimpleField(
        name="aclUsers",
        type=SearchFieldDataType.Collection(SearchFieldDataType.String),
        filterable=True
    ),
    SimpleField(
        name="aclGroups",
        type=SearchFieldDataType.Collection(SearchFieldDataType.String),
        filterable=True
    ),
]

# ベクター検索フィールド（オプション）
if use_vector_search:
    fields.append(
        SearchField(
            name="contentVector",
            type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
            vector_search_dimensions=3072,  # text-embedding-3-large の次元数
            vector_search_profile_name="vector-profile"
        )
    )

# ベクター検索設定
vector_search = None
if use_vector_search:
    vector_search = VectorSearch(
        algorithms=[
            HnswAlgorithmConfiguration(name="hnsw-algorithm")
        ],
        profiles=[
            VectorSearchProfile(
                name="vector-profile",
                algorithm_configuration_name="hnsw-algorithm",
                vectorizer_name="openai-vectorizer"
            )
        ],
        vectorizers=[
            AzureOpenAIVectorizer(
                vectorizer_name="openai-vectorizer",
                parameters=AzureOpenAIVectorizerParameters(
                    resource_url=openai_endpoint,
                    deployment_name=embedding_deployment,
                    api_key=openai_key
                )
            )
        ]
    )

# セマンティック検索設定
semantic_search = SemanticSearch(
    configurations=[
        SemanticConfiguration(
            name="default",
            prioritized_fields=SemanticPrioritizedFields(
                title_field=SemanticField(field_name="title"),
                content_fields=[SemanticField(field_name="content")]
            )
        )
    ]
)

# インデックスの作成
index = SearchIndex(
    name=index_name,
    fields=fields,
    vector_search=vector_search,
    semantic_search=semantic_search
)

try:
    result = index_client.create_or_update_index(index)
    print(f"✅ インデックス '{index_name}' を作成しました")
    print(f"   フィールド数: {len(fields)}")
    print(f"   ベクター検索: {'有効' if use_vector_search else '無効'}")
    print(f"   セマンティック検索: 有効")
except Exception as e:
    print(f"❌ インデックス作成エラー: {e}")
    raise

## 5. SharePoint データソースの作成

SharePoint Online への接続を設定します。

**注意**: SharePoint データソースには、アプリ登録の認証情報が必要です。

In [ ]:
# SharePoint データソースの接続文字列
# 形式: "SharePointOnlineEndpoint=https://{tenant}.sharepoint.com;TenantId={tenantId};ApplicationId={appId};ApplicationSecret={secret}"
connection_string = (
    f"SharePointOnlineEndpoint=https://{sp_hostname};"
    f"TenantId={sp_tenant_id};"
    f"ApplicationId={sp_client_id};"
    f"ApplicationSecret={sp_client_secret}"
)

# データソースコンテナ（サイトとライブラリの指定）
container = SearchIndexerDataContainer(
    name=sp_library,
    query=f"{sp_site_path}"
)

# データソースの作成
datasource = SearchIndexerDataSourceConnection(
    name=datasource_name,
    type="sharepoint",
    connection_string=connection_string,
    container=container,
    description="SharePoint Online document library for AI Search"
)

try:
    result = indexer_client.create_or_update_data_source_connection(datasource)
    print(f"✅ データソース '{datasource_name}' を作成しました")
    print(f"   タイプ: SharePoint")
    print(f"   サイト: https://{sp_hostname}{sp_site_path}")
    print(f"   ライブラリ: {sp_library}")
except Exception as e:
    print(f"❌ データソース作成エラー: {e}")
    print("\nトラブルシューティング:")
    print("  1. アプリ登録の認証情報が正しいか確認")
    print("  2. アプリに Sites.Read.All 権限があるか確認")
    print("  3. 管理者の同意が完了しているか確認")
    print("  4. SharePoint サイトのパスが正しいか確認")
    raise

## 6. スキルセットの作成（オプション）

テキスト抽出、OCR、ベクター化などのスキルを定義します。

**注意**: 基本的なテキスト抽出は SharePoint インデクサに組み込まれているため、
スキルセットなしでも動作します。高度な処理が必要な場合のみ使用します。

In [ ]:
# スキルセットは基本的にオプション
# SharePoint インデクサは標準でテキスト抽出をサポート

skills = []

# テキスト分割スキル（長文を分割）
split_skill = SplitSkill(
    name="text-splitter",
    description="Split text into chunks",
    context="/document/content",
    text_split_mode="pages",
    maximum_page_length=2000,
    page_overlap_length=200,
    inputs=[
        InputFieldMappingEntry(name="text", source="/document/content")
    ],
    outputs=[
        OutputFieldMappingEntry(name="textItems", target_name="pages")
    ]
)
skills.append(split_skill)

# ベクター化スキル（OpenAI 使用時）
if use_vector_search:
    embedding_skill = AzureOpenAIEmbeddingSkill(
        name="embedding-skill",
        description="Generate embeddings using Azure OpenAI",
        context="/document/pages/*",
        resource_url=openai_endpoint,
        deployment_name=embedding_deployment,
        api_key=openai_key,
        inputs=[
            InputFieldMappingEntry(name="text", source="/document/pages/*")
        ],
        outputs=[
            OutputFieldMappingEntry(name="embedding", target_name="vector")
        ]
    )
    skills.append(embedding_skill)

# スキルセットの作成（スキルがある場合のみ）
if skills:
    skillset = SearchIndexerSkillset(
        name=skillset_name,
        description="Skillset for SharePoint indexer",
        skills=skills
    )
    
    try:
        result = indexer_client.create_or_update_skillset(skillset)
        print(f"✅ スキルセット '{skillset_name}' を作成しました")
        print(f"   スキル数: {len(skills)}")
    except Exception as e:
        print(f"⚠️ スキルセット作成エラー: {e}")
        print("   基本的なインデクサはスキルセットなしでも動作します")
        skillset = None
else:
    print("ℹ️ スキルセットなしで続行します（基本的なテキスト抽出のみ）")
    skillset = None

## 7. インデクサの作成と実行

SharePoint からインデックスへの取り込みパイプラインを構成します。

In [ ]:
# フィールドマッピング
# SharePoint のメタデータをインデックスフィールドにマッピング
field_mappings = [
    FieldMapping(source_field_name="metadata_spo_item_weburi", target_field_name="url"),
    FieldMapping(source_field_name="metadata_spo_item_path", target_field_name="path"),
    FieldMapping(source_field_name="metadata_spo_item_name", target_field_name="title"),
    FieldMapping(source_field_name="metadata_spo_item_content_type", target_field_name="contentType"),
    FieldMapping(source_field_name="metadata_spo_item_last_modified", target_field_name="lastModified"),
    FieldMapping(source_field_name="metadata_spo_item_size", target_field_name="size"),
]

# 出力フィールドマッピング（スキルセット使用時）
output_field_mappings = []
if skillset:
    output_field_mappings = [
        FieldMapping(source_field_name="/document/pages/*/vector", target_field_name="contentVector")
    ]

# スケジュール設定（1時間ごと）
schedule = IndexingSchedule(interval="PT1H")  # ISO 8601 duration format

# インデクサの作成
indexer = SearchIndexer(
    name=indexer_name,
    description="SharePoint Online indexer",
    data_source_name=datasource_name,
    target_index_name=index_name,
    skillset_name=skillset_name if skillset else None,
    field_mappings=field_mappings,
    output_field_mappings=output_field_mappings if output_field_mappings else None,
    schedule=schedule,
    parameters={
        "batchSize": 10,
        "maxFailedItems": -1,
        "maxFailedItemsPerBatch": -1,
        "configuration": {
            "dataToExtract": "contentAndMetadata",
            "parsingMode": "default"
        }
    }
)

try:
    result = indexer_client.create_or_update_indexer(indexer)
    print(f"✅ インデクサ '{indexer_name}' を作成しました")
    print(f"   データソース: {datasource_name}")
    print(f"   ターゲットインデックス: {index_name}")
    print(f"   スケジュール: 1時間ごと")
    if skillset:
        print(f"   スキルセット: {skillset_name}")
except Exception as e:
    print(f"❌ インデクサ作成エラー: {e}")
    raise

## 8. インデクサの手動実行

作成したインデクサを手動で実行します。

In [ ]:
print(f"🚀 インデクサ '{indexer_name}' を実行します...\n")

try:
    indexer_client.run_indexer(indexer_name)
    print("✅ インデクサの実行を開始しました")
    print("\nℹ️ 実行状況は次のセルで確認できます")
    print("   大量のドキュメントがある場合、完了まで時間がかかることがあります")
except Exception as e:
    print(f"❌ インデクサ実行エラー: {e}")

## 9. 実行状況の監視

インデクサの実行状況を確認します。

In [ ]:
import time

print("📊 インデクサの状況を確認中...\n")

max_checks = 10
check_interval = 30  # 秒

for i in range(max_checks):
    try:
        status = indexer_client.get_indexer_status(indexer_name)
        
        print(f"\n{'='*80}")
        print(f"確認 #{i+1} / {max_checks}")
        print(f"{'='*80}")
        
        # 全体のステータス
        print(f"ステータス: {status.status}")
        
        # 最新の実行結果
        if status.last_result:
            last_result = status.last_result
            print(f"\n最新の実行:")
            print(f"  状態: {last_result.status}")
            print(f"  開始: {last_result.start_time}")
            print(f"  終了: {last_result.end_time}")
            
            if hasattr(last_result, 'item_count'):
                print(f"  処理アイテム数: {last_result.item_count}")
            if hasattr(last_result, 'failed_item_count'):
                print(f"  失敗アイテム数: {last_result.failed_item_count}")
            
            # エラーがある場合
            if last_result.errors:
                print(f"\n⚠️ エラー ({len(last_result.errors)} 件):")
                for error in last_result.errors[:3]:  # 最大3件表示
                    print(f"  - {error.error_message}")
            
            # 警告がある場合
            if last_result.warnings:
                print(f"\n⚠️ 警告 ({len(last_result.warnings)} 件):")
                for warning in last_result.warnings[:3]:  # 最大3件表示
                    print(f"  - {warning.message}")
            
            # 完了判定
            if last_result.status in ["success", "transientFailure"]:
                print("\n✅ インデクサの実行が完了しました")
                break
            elif last_result.status == "inProgress":
                print(f"\n⏳ 実行中... {check_interval}秒後に再確認します")
            else:
                print(f"\n❌ 実行が失敗しました: {last_result.status}")
                break
        else:
            print("\nℹ️ まだ実行結果がありません")
        
        # 次のチェックまで待機
        if i < max_checks - 1:
            time.sleep(check_interval)
    
    except Exception as e:
        print(f"❌ ステータス取得エラー: {e}")
        break

print("\nℹ️ 詳細な監視は B2_indexer_monitor_troubleshoot.ipynb を参照してください")

## 10. インデックス内容の確認

In [ ]:
from azure.search.documents import SearchClient

# Search クライアントの作成
search_client = SearchClient(
    endpoint=search_endpoint,
    index_name=index_name,
    credential=credential
)

print("📊 インデックスの内容を確認中...\n")

try:
    # ドキュメント数の取得
    results = search_client.search(search_text="*", top=1, include_total_count=True)
    total_docs = results.get_count()
    
    print(f"✅ インデックス内のドキュメント数: {total_docs}")
    
    if total_docs > 0:
        # サンプルドキュメントの表示
        print("\n📄 サンプルドキュメント (最大3件):\n")
        
        sample_results = search_client.search(
            search_text="*",
            top=3,
            select=["id", "title", "contentType", "lastModified", "url"]
        )
        
        for i, doc in enumerate(sample_results, 1):
            print(f"{i}. {doc.get('title', 'タイトルなし')}")
            print(f"   種類: {doc.get('contentType', 'N/A')}")
            print(f"   URL: {doc.get('url', 'N/A')}")
            print()
    else:
        print("\n⚠️ まだドキュメントが取り込まれていません")
        print("   インデクサの実行が完了するまでお待ちください")

except Exception as e:
    print(f"❌ インデックス確認エラー: {e}")

## まとめ

このノートブックでは、SharePoint インデクサを使った自動取り込みを設定しました:

1. ✅ **インデックス作成**: 検索用スキーマの定義
2. ✅ **データソース作成**: SharePoint への接続
3. ✅ **スキルセット作成**: テキスト抽出・ベクター化（オプション）
4. ✅ **インデクサ作成**: 自動取り込みパイプライン
5. ✅ **初回実行**: データの取り込み開始

### 次のステップ

- **B2_indexer_monitor_troubleshoot.ipynb**: 詳細な監視とトラブルシューティング
- **B3_query_and_acl_demo.ipynb**: 検索とACLフィルタのデモ

### インデクサの管理

- **スケジュール実行**: 1時間ごとに自動で差分同期
- **手動実行**: `indexer_client.run_indexer(indexer_name)`
- **一時停止**: インデクサを停止する場合は Azure Portal から

### 参考リソース

- [SharePoint Indexer ドキュメント](https://learn.microsoft.com/azure/search/search-howto-index-sharepoint-online)
- [インデクサの概要](https://learn.microsoft.com/azure/search/search-indexer-overview)
- [スキルセット](https://learn.microsoft.com/azure/search/cognitive-search-working-with-skillsets)